In [6]:
import numpy as np
import geopy
import pandas as pd
import math
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

In [4]:
infers1 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer1_horizon_0.csv')
infers2 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer1_horizon_180.csv')
infers3 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer1_vertical_270.csv')
infers4 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer1_vertical_90.csv')
infers5 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer2_horizon_0.csv')
infers6 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer2_horizon_180.csv')
infers7 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer2_vertical_270.csv')
infers8 = pd.read_csv('/Users/richardryu/Desktop/Cantwo/w210/output_yolo_infer2_vertical_90.csv')

In [7]:
search = SearchEngine()

def csv_par(df):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=3)
        df.loc[i, 'zipcode'] = result[0].zipcode
    return df

# lat/long rounded to 4 decimals
def csv_par2(df):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=3)
        df.loc[i, 'zipcode'] = result[0].zipcode
        df.loc[i, 'lat'] = df.loc[i, 'lat'].astype(float).round(4)
        df.loc[i, 'long'] = df.loc[i, 'long'].astype(float).round(4)
    return df

#7/5 adjusting coordinates by 'xcenter' position [L, C, R]. 0.013 mile = ~20 meters = 0.0002
# currently only accounts for the 0 angle. need to account for 'angle'
def csv_par3(df, delta):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=3)
        df.loc[i, 'zipcode'] = result[0].zipcode
        df.loc[i, 'lat'] = df.loc[i, 'lat'].astype(float).round(4)
        df.loc[i, 'long'] = df.loc[i, 'long'].astype(float).round(4)
        if df.loc[i, 'xcenter'] <= 0.333333:
            df.loc[i, 'detect_loc'] = 'L'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] + delta/2
            df.loc[i, 'new_long'] = df.loc[i, 'long']
        elif df.loc[i, 'xcenter'] > 0.666666:
            df.loc[i, 'detect_loc'] = 'R'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat']
            df.loc[i, 'new_long'] = df.loc[i, 'long']
        else:
            df.loc[i, 'detect_loc'] = 'C'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] - delta/2
            df.loc[i, 'new_long'] = df.loc[i, 'long']
    return df

#7/5 adjusting coordinates by 'xcenter' position [L, C, R]. 0.013 mile = ~20 meters = 0.0002
# Rachael's version
def csv_par4(df, delta):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=3)
        df.loc[i, 'zipcode'] = result[0].zipcode
        df.loc[i, 'lat'] = df.loc[i, 'lat'].astype(float).round(4)
        df.loc[i, 'long'] = df.loc[i, 'long'].astype(float).round(4)
        df.loc[i, 'new_lat'] = df.loc[i, 'lat'] + (delta * math.cos((math.pi / 180)*float(df.loc[i, 'angle'])))
        df.loc[i, 'new_long'] = df.loc[i, 'long'] + (delta * math.sin((math.pi / 180)*float(df.loc[i, 'angle'])))
    return df

In [8]:
# 7/1 save df by zipcode in csv
def zip_df(df):
    for i in df["zipcode"].unique():
        temp_df = df[df["zipcode"] == i]
        temp_df.to_csv(i + '.csv', encoding = 'utf-8', index = False)

In [9]:
output1 = csv_par(infers1)
output2 = csv_par(infers2)
output3 = csv_par(infers3)
output4 = csv_par(infers4)
output5 = csv_par(infers5)
output6 = csv_par(infers6)
output7 = csv_par(infers7)
output8 = csv_par(infers8)

In [10]:
final_df = pd.concat([output1, output2, output3, output4, output5, output6, output7, output8], ignore_index=True)

In [15]:
zip_df(final_df)

In [16]:
# rounded version
r_output1 = csv_par2(infers1)
r_output2 = csv_par2(infers2)
r_output3 = csv_par2(infers3)
r_output4 = csv_par2(infers4)
r_output5 = csv_par2(infers5)
r_output6 = csv_par2(infers6)
r_output7 = csv_par2(infers7)
r_output8 = csv_par2(infers8)

In [17]:
final_df2 = pd.concat([r_output1, r_output2, r_output3, r_output4, r_output5, r_output6, r_output7, r_output8], ignore_index=True)

In [18]:
zip_df(final_df2)

In [20]:
# angle modified coordinates
a_output1 = csv_par4(infers1, 0.0002)
a_output2 = csv_par4(infers2, 0.0002)
a_output3 = csv_par4(infers3, 0.0002)
a_output4 = csv_par4(infers4, 0.0002)
a_output5 = csv_par4(infers5, 0.0002)
a_output6 = csv_par4(infers6, 0.0002)
a_output7 = csv_par4(infers7, 0.0002)
a_output8 = csv_par4(infers8, 0.0002)

In [21]:
final_df3 = pd.concat([a_output1, a_output2, a_output3, a_output4, a_output5, a_output6, a_output7, a_output8], ignore_index=True)

In [22]:
zip_df(final_df3)